# Using the simulator

In this notebook you will learn how to use the simulator to test out methods.

## Setting up a connection with the robot

As described in the [basic liquid handling tutorial](basic), we will use the {class}`~pyhamilton.liquid_handling.liquid_handler.LiquidHandler` class to control the robot. This time, however, instead of using the Hamilton {class}`~pyhamilton.liquid_handling.backends.hamilton.STAR` backend, we are using virtual the {class}`~pyhamilton.liquid_handling.backends.simulation.simulation.SimulationBackend` backend. This means that liquid handling will work exactly the same, but the commands are sent to the simulator instead of a real physical robot.

In [1]:
from pyhamilton.liquid_handling import LiquidHandler
from pyhamilton.liquid_handling.backends.simulation.simulation import SimulationBackend

In [2]:
from pyhamilton.liquid_handling.resources import (
    TIP_CAR_480_A00,
    PLT_CAR_L5AC_A00,
    Cos_96_DW_1mL,
    STF_L
)

sb = SimulationBackend()
lh = LiquidHandler(backend=sb)

Calling {func}`~pyhamilton.liquid_handling.backends.simulation.SimulationBackend.setup` will set up the simulation server and open it in a new browser tab.

In [3]:
lh.setup()

INFO:websockets.server:server listening on 127.0.0.1:2121
INFO:pyhamilton.liquid_handling.backends.simulation.simulation:Simulation server started at http://127.0.0.1:2121
INFO:pyhamilton.liquid_handling.backends.simulation.simulation:File server started at http://127.0.0.1:1337


For the optimal experience, we recommend that you run the notebook and simulator side by side.

![The empty simulator](./img/simulator/empty.png)

## Assigning plates and tips

With the simulator, {func}`~assigning resources <pyhamilton.liquid_handling.LiquidHandler.assign_resource>` has the additional affect of placing the resources on the simulated deck. They will appear immediately.

In [4]:
tip_car = TIP_CAR_480_A00(name='tip carrier')
for i in range(2):
    tip_car[i] = STF_L(name=f'tip_{i}')

In [5]:
lh.assign_resource(tip_car, rails=1, replace=True)

When you set or delete subresources in a {class}`~pyhamilton.liquid_handling.resources.abstract.Carrier`, that will immediately be reflected in the UI.

In [6]:
tip_car[2] = STF_L(name=f'tip_{2}')

In [7]:
tip_car[3] = STF_L(name=f'tip_{3}')

In [8]:
del tip_car[3]

In [9]:
tip_car[4] = STF_L(name=f'tip{4}')

In [10]:
tip_car[4] = None

In [11]:
tip_car[3] = None

In [12]:
tip_car[3] = STF_L(name=f'tip_{3}')

In [13]:
tip_car[4] = STF_L(name=f'tip_{4}')

In [14]:
plt_car = PLT_CAR_L5AC_A00(name='pre')
for i in range(5):
    plt_car[i] = Cos_96_DW_1mL(name=f'plate_{i}')

In [15]:
lh.assign_resource(plt_car, rails=9, replace=True)

![The simulator after the resources have been assigned](./img/simulator/assignment.png)

## Build the deck layout: placing resources

Where you would manually place the resources like tips and liquid on the deck when using a physical system, with the Simulator you can add them using code.

### Tips

Let's {func}`~pyhamilton.liquid_handling.backends.simulation.SimulationBackend.fill_tips` the tip containers at locations `0` and `1`.

In [16]:
sb.fill_tips(tip_car[0])

In [17]:
sb.fill_tips(tip_car[1])

When you no longer need tips, use {func}`~pyhamilton.liquid_handling.backends.simulation.SimulationBackend.remove_tips` to remove them.

In [18]:
sb.fill_tips(tip_car[4])
sb.remove_tips(tip_car[4], pattern=[[True]*6 + [False]*6]*8)

In [19]:
sb.place_tips(tip_car[3], pattern=[[True, False]*6]*8)

In [20]:
sb.place_tips(tip_car[2], pattern=[[True, True, False, False]*3]*8)

### Liquids

Adding liquid to wells works similarly. You can use {func}`~pyhamilton.liquid_handling.backends.simulation.SimulationBackend.adjust_well_volume` to adjust the volume of individual wells in each resource. Note that the opacity of the well matches the volume of the well.

In [21]:
sb.adjust_well_volume(plt_car[2], pattern=[[100, 800]*6]*8)

In [22]:
sb.adjust_well_volume(plt_car[1], pattern=[[1000]*12]*8)

In [23]:
sb.adjust_well_volume(plt_car[0], pattern=[[0]*12]*8)

![Simulator after the tips have been placed and the volumes adjusted](./img/simulator/resources.png)

## Liquid handling

Once the layout is complete, you can run the same commands as described in the [basic liquid handling tutorial](basic).

### Picking up tips

In [24]:
lh.pickup_tips('tip_0', "A1", "B2", "C3", "D4")

In [25]:
lh.discard_tips('tip_0', "A1", "B2", "C3", "D4")

### Aspirating and dispensing

In [26]:
lh.pickup_tips('tip_0', "A1")

In [27]:
lh.aspirate(plt_car[2], ("A2", 300))

In [28]:
lh.dispense(plt_car[4], ("A2", 300))

In [29]:
lh.discard_tips('tip_0', "A1")

### Aspirating using CoRe 96

The CoRe 96 head supports liquid handling operations for 96 channels at once. Here's how to use:

- {func}`~pyhamilton.liquid_handling.LiquidHandler.pickup_tips96` for picking up 96 tips;
- {func}`~pyhamilton.liquid_handling.LiquidHandler.aspirate96` for aspirating liquid from the deck, where the `pattern` array controls which wells are aspirated from;
- {func}`~pyhamilton.liquid_handling.LiquidHandler.dispense96` for dispensing liquid to the deck, where the `pattern` array controls which wells are dispensed to, and;
- {func}`~pyhamilton.liquid_handling.LiquidHandler.discard_tips96` for discarding tips to the resource.


In [30]:
lh.pickup_tips96(tip_car[0])

In [31]:
lh.aspirate96(plt_car[1], [[True] * 12] * 8, volume=400)

In [32]:
lh.dispense96(plt_car[0], [[True] * 12] * 8, volume=400)

In [33]:
lh.aspirate96(plt_car[1], "A1:H12", volume=400)

In [34]:
lh.dispense96(plt_car[0], "A1:H12", volume=400)

In [35]:
lh.discard_tips96(tip_car[0])

![The simulator after the liquid handling operations completed](./img/simulator/after_lh.png)

## Shutting down

When you're done, remember to shut down the simulator by calling {func}`~pyhamilton.liquid_handling.LiquidHandler.stop`.

In [39]:
lh.stop()

INFO:websockets.server:server closing
INFO:websockets.server:connection closed
